In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
def ex_tag_detail(category_code, page):
    """
    특정 카테고리의 기사 링크를 추출하는 함수.
    
    Args:
        category_code (int): 카테고리 코드 (금융=259, 증권=258, 등)
        page (int): 페이지 번호
    
    Returns:
        list: 기사 링크 리스트
    """
    # Naver 뉴스 URL
    url = f"https://news.naver.com/breakingnews/section/101/{category_code}?page={page}"
    
    # User-Agent 설정
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
        )
    }
    
    #HTML 요청
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 기사 링크 추출
    articles = soup.select("a[href*='article']")
    
    # 링크 저장
    tag_lst = [a["href"] for a in articles if "href" in a.attrs]
    
    return tag_lst


In [3]:
# 상세 카테고리 코드
category_codes = {
    "finance": 259,    # 금융
    "stock": 258,      # 증권
    "industry": 261,   # 산업
    "sme": 771,        # 중소기업
    "real_estate": 260 # 부동산
}

all_category_links = {}  # 모든 카테고리별 링크 저장

# 각 카테고리에 대해 링크 크롤링
for category, code in category_codes.items():
    print(f"크롤링 중: {category}")
    category_links = []
    for page in range(1, 3):  # 페이지 범위 조정 가능
        links = ex_tag_detail(code, page)
        category_links.extend(links)
    all_category_links[code] = category_links


크롤링 중: finance
크롤링 중: stock
크롤링 중: industry
크롤링 중: sme
크롤링 중: real_estate


In [4]:
# 크롤링 결과 출력
for category, links in all_category_links.items():
    print(f"[{category}] 총 {len(links)}개 링크 수집 완료")

[259] 총 386개 링크 수집 완료
[258] 총 384개 링크 수집 완료
[261] 총 386개 링크 수집 완료
[771] 총 386개 링크 수집 완료
[260] 총 384개 링크 수집 완료


In [5]:
all_category_links

{259: ['https://n.news.naver.com/mnews/article/018/0005901978',
  'https://n.news.naver.com/mnews/article/018/0005901978',
  'https://n.news.naver.com/mnews/article/comment/018/0005901978',
  'https://n.news.naver.com/mnews/article/243/0000069323',
  'https://n.news.naver.com/mnews/article/243/0000069323',
  'https://n.news.naver.com/mnews/article/comment/243/0000069323',
  'https://n.news.naver.com/mnews/article/016/0002399244',
  'https://n.news.naver.com/mnews/article/016/0002399244',
  'https://n.news.naver.com/mnews/article/comment/016/0002399244',
  'https://n.news.naver.com/mnews/article/016/0002399242',
  'https://n.news.naver.com/mnews/article/016/0002399242',
  'https://n.news.naver.com/mnews/article/comment/016/0002399242',
  'https://n.news.naver.com/mnews/article/003/0012950069',
  'https://n.news.naver.com/mnews/article/003/0012950069',
  'https://n.news.naver.com/mnews/article/comment/003/0012950069',
  'https://n.news.naver.com/mnews/article/003/0012950068',
  'https://

In [6]:
from tqdm import tqdm

def remove_duplicates(all_category_links):
    """
    카테고리별 링크 리스트에서 중복을 제거하는 함수.
    
    Args:
        all_category_links(dict): 카테고리 코드(key)와 해당 카테고리의 링크 리스트(value).
        
    Returns:
        dict: 중복이 제거된 링크 리스트가 포함된 딕셔너리.
    """
    unique_links = {}
    
    for category_code, links in tqdm(all_category_links.items(), desc="Removing Duplicates"):
        # 중복 제거
        unique_links[category_code] = list(set(links))
    
    return unique_links

# 중복 제거 실행
unique_category_links = remove_duplicates(all_category_links)

unique_category_links

Removing Duplicates: 100%|█████| 5/5 [00:00<00:00, 4964.85it/s]


{259: ['https://n.news.naver.com/mnews/ranking/article/014/0005279135?ntype=RANKING',
  'https://n.news.naver.com/mnews/ranking/article/016/0002399005?ntype=RANKING',
  'https://n.news.naver.com/mnews/article/277/0005513738',
  'https://n.news.naver.com/mnews/ranking/article/055/0001213469?ntype=RANKING',
  'https://n.news.naver.com/mnews/article/comment/014/0005279268',
  'https://n.news.naver.com/mnews/article/comment/277/0005513769',
  'https://n.news.naver.com/mnews/ranking/article/648/0000031386?ntype=RANKING',
  'https://n.news.naver.com/mnews/article/comment/029/0002921286',
  'https://n.news.naver.com/mnews/ranking/article/437/0000421633?ntype=RANKING',
  'https://n.news.naver.com/mnews/article/031/0000891604',
  'https://n.news.naver.com/mnews/article/comment/421/0007954979',
  'https://n.news.naver.com/mnews/ranking/article/015/0005067442?ntype=RANKING',
  'https://n.news.naver.com/mnews/ranking/article/293/0000061434?ntype=RANKING',
  'https://n.news.naver.com/mnews/ranking/

In [7]:
def art_crawl(all_hrefs, category_code, index):
    """
    카테고리 코드와 링크 인덱스를 사용하여 기사 제목, 날짜, 본문을 크롤링.
    
    Args: 
        all_hrefs(dict): 각 카테고리별로 수집된 링크 딕셔너리
        category_code(int): 카테고리 코드 (e.g., 금융=259, 증권=258)
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사 제목, 날짜, 본문을 포함한 딕셔너리
    """
    art_dic = {}
    
    ## 1.
    # CSS Selector 설정
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    # URL 가져오기
    url = all_hrefs[category_code][index]
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
        )
    }
    response = requests.get(url, headers=headers)
    # lxml -> html.parser
    soup = BeautifulSoup(response.text, "html.parser")
    
    
    ## 2.
    # 제목 수집
    title = soup.select_one(title_selector)
    art_dic["title"] = title.text.strip() if title else "No title"
    
    # 날짜 수집
    date = soup.select_one(date_selector)
    art_dic["date"] = date.text.strip() if date else "No date"
    
    # 본문 수집
    main = soup.select_one(main_selector)
    art_dic["main"] = main.text.strip() if main else "No content"
    
    return art_dic

# 카테고리 코드 리스트
category_codes = [259, 258, 261, 771, 260]  # 금융, 증권, 산업, 중소기업, 부동산
artdic_lst = []

# 각 카테고리 코드의 기사 데이터를 수집
for category_code in tqdm(category_codes, desc="카테고리 진행"):
    links = unique_category_links[category_code]
    for i in tqdm(range(len(links)), desc=f"{category_code} 진행"):
        try:
            # 기사 데이터 크롤링
            art_dic = art_crawl(unique_category_links, category_code, i)
            art_dic["category_code"] = category_code
            art_dic["url"] = links[i]
            artdic_lst.append(art_dic)
        except Exception as e:
            print(f"Error in {category_code} index {i}: {e}")


259 진행:  80%|███████████▏  | 124/155 [02:59<00:51,  1.65s/it]


258 진행:  60%|█████████      | 93/155 [02:22<01:44,  1.68s/it]


261 진행:  38%|█████▋         | 62/163 [01:14<02:00,  1.19s/it]


771 진행:  15%|██▏            | 23/155 [01:03<04:46,  2.17s/it]


771 진행:  96%|█████████████▍| 149/155 [03:43<00:07,  1.32s/it]


260 진행:  76%|██████████▋   | 118/155 [03:48<01:16,  2.08s/it]


카테고리 진행: 100%|████████████| 5/5 [19:56<00:00, 239.21s/it]


In [8]:
artdic_lst

[{'title': '"57억 내놔" 남편이 밀어 34m 절벽서 추락한 中여성, 생존 후 한 말',
  'date': '2024.12.09. 오전 8:09',
  'main': '5년 전 임신 3개월 중이던 아내 살인미수기적생환한 아내, 이혼 요구 위자료 청구 \n\n\n\n관련 시각물 - SCMP 갈무리 /사진=뉴스1 [파이낸셜뉴스] 5년 전 태국 여행 중 남편이 절벽에서 밀어 구사일생으로 살아난 중국 여성이 남편에게 이혼을 요구하며 위자료로 3000만위안(약 57억원)을 청구했다.   6일(현지시간) 홍콩 사우스차이나모닝포스트(SCMP)는 2019년 태국 북동부 파탐 국립공원에서 휴가를 보내던 중 남편이 절벽에서 밀어 34m 아래로 떨어졌던 중국 여성 왕난(37)의 소식을 전했다.   왕씨는 살인 미수 혐의 등으로 33년 4개월의 징역형을 선고받아 태국 교도소에 수감 중인 전남편 위샤오둥(38)과 아직 법적 부부 관계인데, 이혼과 위자료를 요구한 사실이 알려지며 화제가 되고 있다.   당시 임신 3개월 차였던 왕씨는 이 사고로 뱃속의 아이를 잃었다. 뿐만 아니라 17군데 골절상을 입고 5번의 수술을 받았으며, 몸에 100개 이상의 쇠침을 박고 3년 동안 휠체어를 타야 했다.   그러나 왕씨는 피나는 재활 훈련 끝에 지난해 걸을 수 있게 됐고, 완쾌한 뒤에는 가장 먼저 자신을 구조한 태국 구조대원과 지역 경찰을 찾아가 감사 인사를 전했다. 또한 지난해 9월 체외수정으로 아들을 낳는 등 후일담으로 여러 차례 언론에 보도된 바 있다.   왕씨가 정식으로 이혼 소송을 제기한 것도 바로 이 아들 때문이다. 아직 법적 부부여서 아들이 남편의 성을 따라야 하자 최근 정식으로 이혼 소송을 제기한 것. 그러나 복역 중인 남편이 재판에 참석할 수 없어 소송에 어려움을 겪고 있는 상태인 것으로 알려졌다. #위자료 #추락사고 #이혼소송 #절벽',
  'category_code': 259,
  'url': 'https://n.news.naver.com/mnews/ranking/a

In [9]:
# DataFrame 생성
art_df = pd.DataFrame(artdic_lst)
art_df

,title,date,main,category_code,url
0,"""57억 내놔"" 남편이 밀어 34m 절벽서 추락한 中여성, 생존 후 한 말",2024.12.09. 오전 8:09,"5년 전 임신 3개월 중이던 아내 살인미수기적생환한 아내, 이혼 요구 위자료 청구 ...",259,https://n.news.naver.com/mnews/ranking/article...
1,“기안84 너무 믿었다가” 시청률 0% 쓴맛…예능 만들었다 ‘낭패’,2024.12.08. 오후 8:41,"[사진, ENA][헤럴드경제= 박영훈 기자] “너무 식상한가?”LG유플러스와 KT가...",259,https://n.news.naver.com/mnews/ranking/article...
2,"[특징주]금융주, 정치적 불확실성에 일제히 하락",2024.12.09. 오전 10:06,원·달러 환율 6.8원 상승 개장밸류업 정책 동력 상실\n\n\n\n외국인 투자자 ...,259,https://n.news.naver.com/mnews/article/277/000...
3,"[정치쇼] 與 김근식 ""尹 탄핵에 의한 직무정지가 가장 질서 있는 퇴진""",2024.12.09. 오전 8:05,"- 탄핵안 기권? 與, 尹과 함께 민심 쓰나미에 쓸려갈라\n- 내란동조 정당으로 낙...",259,https://n.news.naver.com/mnews/ranking/article...
4,"노무라증권 ""원달러 환율, 내년 5월 1500원까지 상승""",2024.12.09. 오전 10:17,No content,259,https://n.news.naver.com/mnews/article/comment...
...,...,...,...,...,...
778,"한국계 첫 미국 연방 상원의원 앤디 김, 의정 활동 조기에 시작",2024.12.09. 오전 6:33,"뉴저지 주지사, 임시 상원의원 사퇴에 따라 후임으로 임명""상원에서 일하게 돼 영광…...",260,https://n.news.naver.com/mnews/ranking/article...
779,"삼성물산, 글로벌 초고압직류송전 시장 공략…히타치에너지와 업무협약",2024.12.09. 오전 10:31,No content,260,https://n.news.naver.com/mnews/article/comment...
780,"[속보] 민주당, '내란 특검법·김건희 특검법' 국회 제출",2024.12.09. 오전 9:25,더불어민주당이 9일 윤석열 대통령 등의 내란 혐의에 대한 '내란 특검법'과 지난 7...,260,https://n.news.naver.com/mnews/ranking/article...
781,"서울 85㎡·9억원 이하 아파트 거래, 9개월 만에 최고치 기록",2024.12.09. 오전 10:35,6억원 초과 9억원 이하 거래 47%로 가장 많아 서울 전용면적 85㎡...,260,https://n.news.naver.com/mnews/article/366/000...


In [10]:
# 결과를 CSV로 저장
art_df.to_csv("category_article_df.csv", index=False, encoding="utf-8-sig")